In [ ]:
# El obejtivo de este notebook sera definir las variables que necesitamos para los modelos: 
# el de clasificacion y el de regresion
# y comenzar a testear las conexiones con la api para despues generar el .py de get_data




# Modelo de clasificacion: Que Equipo Va a Ganar?

In [ ]:
# Identificador único del juego.
# Fecha del partido (para calcular el descanso).
# ID de los equipos (home y away)
# Los puntos finales anotados por cada equipo (nos ayudara a crear la target column).
# is_home?

# Para Home y Away
# FGM	Tiros de campo anotados.
# FGA	Tiros de campo intentados.
# FG3M	Tiros de 3 puntos anotados.
# OREB	Rebotes ofensivos.
# ODREB	Rebotes defensivos.
# AST	Asistencias.
# TOV	Pérdidas de balón.
# FTM / FTA	Tiros libres anotados / intentados. (Tasa de Tiros Libres)

# Me falta agregar columnas que puedan afectar psicologicamente como finales etc, investigarlas

# Is_Playoffs
# Is_Finals
# Game_In_Series
# Series_Is_Elimination
# Win_Streak
# Days_Since_Loss
# Time_of_Day_Factor



# La idea sera con estas columnas hacer nuestro FE, el modelo no va a predecir al ganador basado en stats 
# que aun no conocemos (data lekeage) usaremos lags y stadisticas acumuladas hasta la fecha mas recientes antes del juego
# asi el modelo debe aprender a predecir al equipo ganandor, basandose en el comportamiento reciente de los 2 equipos





# Api Conection

In [ ]:
import requests
import pandas as pd 

# Clave personal de API
api_key = "886eff79607baa34813985f6201a4e68"

# URL base (v1 para Basketball)
base_url = "https://v1.basketball.api-sports.io"

# Headers comunes para todas las peticiones
headers = {
    "x-rapidapi-key": api_key,
    "x-rapidapi-host": "v1.basketball.api-sports.io"
} 

# API GENERAL DOCUMENTATION : https://api-sports.io/documentation/basketball/v1
# PYTHON CONECTION: https://api-sports.io/documentation/basketball/v1#section/Sample-Scripts/Python

# Endpoints

In [108]:
# LEAGUES ENDPOINTS DOC: https://api-sports.io/documentation/basketball/v1#tag/Leagues

# We need to know what id NBA league has

url = f"{base_url}/leagues" # '/leagues' is the endpoint

# request to the api
res = requests.get(url, headers=headers)
# we convert the json response to a python dict
data = res.json()
# the response section od the dict is a list od diccionaries with the asnwer, we want that list in a pandas df
leagues_df  = pd.json_normalize(data["response"])
# visualize the df
display(leagues_df[leagues_df['name'].str.contains('NBA')].head(3))

# Now we know that NBA is league = 12
league = 12 

,id,name,type,logo,seasons,country.id,country.name,country.code,country.flag
379,12,NBA,League,https://media.api-sports.io/basketball/leagues...,"[{'season': '2021-2022', 'start': '2021-10-03'...",5,USA,US,https://media.api-sports.io/flags/us.svg
380,20,NBA - G League,League,https://media.api-sports.io/basketball/leagues...,"[{'season': '2016-2017', 'start': '2016-11-12'...",5,USA,US,https://media.api-sports.io/flags/us.svg
381,17,NBA - Las Vegas Summer League,League,https://media.api-sports.io/basketball/leagues...,"[{'season': 2022, 'start': '2022-07-08', 'end'...",5,USA,US,https://media.api-sports.io/flags/us.svg


In [111]:
# TEAMS ENDPOINT DOC: https://api-sports.io/documentation/basketball/v1#tag/Teams
# We want a list of the NBA teams unqiue id's 

url = f"{base_url}/teams" # '/teams' is the API endpoint
seasons = ['2021-2022', '2022-2023'] # We need to iterate this list of seasons

# Empty df to concatenate the results
teams_df = pd.DataFrame()

for season in seasons:
    params = {'league':league, 'season': season} 
    res = requests.get(url, headers = headers, params = params) 
    data = res.json()
    df = pd.json_normalize(data['response'])
    df['season'] = season
    teams_df = pd.concat([teams_df, df])

teams_df.head()

,id,name,logo,nationnal,country.id,country.name,country.code,country.flag,season
0,132,Atlanta Hawks,https://media.api-sports.io/basketball/teams/1...,False,5,USA,US,https://media.api-sports.io/flags/us.svg,2021-2022
1,133,Boston Celtics,https://media.api-sports.io/basketball/teams/1...,False,5,USA,US,https://media.api-sports.io/flags/us.svg,2021-2022
2,134,Brooklyn Nets,https://media.api-sports.io/basketball/teams/1...,False,5,USA,US,https://media.api-sports.io/flags/us.svg,2021-2022
3,135,Charlotte Hornets,https://media.api-sports.io/basketball/teams/1...,False,5,USA,US,https://media.api-sports.io/flags/us.svg,2021-2022
4,136,Chicago Bulls,https://media.api-sports.io/basketball/teams/1...,False,5,USA,US,https://media.api-sports.io/flags/us.svg,2021-2022


# Modelo de Regresion: Cuantos puntos habra?